In [1]:
import pandas as pd
import utils
# Read the CSV file from Google Drive
filename = 'Data//train_data.csv'
df = pd.read_csv(filename)

ImportError: DLL load failed while importing ft2font: 지정된 모듈을 찾을 수 없습니다.

In [ ]:
# Print the DataFrame
print(df.info())

: 

In [ ]:
sample_df=df['fog_train.vis1']<20000

: 

In [ ]:
# 각 클래스별로 10% 샘플링
sampled_df = df.groupby('fog_train.class', group_keys=False).apply(lambda x: x.sample(frac=0.1))

# 샘플링된 데이터 저장
sampled_df.to_csv('Data//sampled_train.csv', index=False)

: 

In [ ]:
# 결과 확인
print(sampled_df)

: 

In [ ]:
# 데이터 프레임 정보 확인
sampled_df_info = sampled_df.info()

# 데이터 통계 요약
sampled_df_description = sampled_df.describe()

print(sampled_df_info, sampled_df_description)

: 

In [ ]:
# 'Unnamed: 0' 열과 'fog_train.vis1' 열 삭제 -> vis1을 예측하는 class 문제로 바꾸기 -> 오히려 class 데이터를 없애야한다.
sampled_df_cleaned = sampled_df.drop(columns=['Unnamed: 0','fog_train.ws10_deg'])
# fog_train.ws10_deg 너무 낮아서 일단 없애보기

# 결과 확인
# print(sampled_df_cleaned)

: 

In [ ]:
# 이상치 처리
cleaned_data = utils.handle_outliers(sampled_df_cleaned)

: 

In [ ]:
# 결과 확인
cleaned_data_info = cleaned_data.info()
cleaned_data_description = cleaned_data.describe()

print(cleaned_data_info, cleaned_data_description)

: 

In [ ]:
# 'fog_train.year' 열의 값을 연도 값으로 변환
cleaned_data['fog_train.year'] = utils.convert_year(cleaned_data['fog_train.year'])
cleaned_data['fog_train.year'] = cleaned_data['fog_train.year'].astype(int)

: 

In [ ]:
# 'fog_train.month'를 기반으로 계절 열 추가
cleaned_data['season'] = cleaned_data['fog_train.month'].apply(utils.get_season)

: 

In [ ]:
# 'fog_train.day'를 카테고리로 변환하여 새로운 열 추가
cleaned_data['day_category'] = cleaned_data['fog_train.day'].apply(utils.categorize_day)

: 

In [ ]:
# 측정소 ID의 앞자리와 뒷자리를 분리하여 새로운 열 추가
cleaned_data['region'] = cleaned_data['fog_train.stn_id'].str[0]
# cleaned_data['station'] = cleaned_data['fog_train.stn_id'].str[1]

: 

In [ ]:
# 'fog_train.year', 'fog_train.month', 'fog_train.day', 'fog_train.stn_id' 열 삭제
cleaned_data.drop(columns=['fog_train.day', 'fog_train.stn_id','fog_train.minute','fog_train.class'], inplace=True)

: 

In [ ]:
# 결과 확인
cleaned_data.info()
cleaned_data.head()

: 

In [ ]:
# 원-핫 인코딩
X_ohe_encoded = cleaned_data.copy()
X_ohe_encoded = pd.get_dummies(X_ohe_encoded, columns=['fog_train.year','region', 'season', 'day_category','fog_train.re'])

X_ohe_encoded.info()

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_label_encoded= cleaned_data.copy()

# 레이블 인코딩
label_encoders = {}
for column in ['fog_train.year','region', 'season', 'day_category']:
    le = LabelEncoder()
    X_label_encoded[column] = le.fit_transform(X_label_encoded[column])
    label_encoders[column] = le

X_label_encoded.head()

: 

# PCA 적용 

In [ ]:
# from sklearn.decomposition import PCA
# import numpy as np

# # PCA 모델 초기화 (설명 분산 비율의 95%를 유지하도록)
# pca = PCA(n_components=0.95)
# X_pca = pca.fit_transform(X_ohe_encoded.drop(columns=['fog_train.vis1']))
# y_pca = X_ohe_encoded['fog_train.vis1']
# X_pca = pd.DataFrame(X_pca, index=X_ohe_encoded.index)

# # PCA 결과
# print(f"Original number of features: {X_ohe_encoded.shape[1]}")
# print(f"Reduced number of features: {X_pca.shape[1]}")

# # 주성분이 설명하는 분산 비율
# explained_variance = pca.explained_variance_ratio_
# print(f"Explained variance by each component: {explained_variance}")
# print(f"Cumulative explained variance: {np.cumsum(explained_variance)}")

: 

In [ ]:
# import matplotlib.pyplot as plt

# # 주성분이 설명하는 분산 비율 시각화
# plt.figure(figsize=(10, 5))
# plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.5, align='center', label='individual explained variance')
# plt.step(range(1, len(np.cumsum(explained_variance)) + 1), np.cumsum(explained_variance), where='mid', label='cumulative explained variance')
# plt.ylabel('Explained variance ratio')
# plt.xlabel('Principal components')
# plt.legend(loc='best')
# plt.tight_layout()
# plt.show()

: 

In [ ]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.model_selection import train_test_split

# H2O 서버 시작
h2o.init(
    nthreads=-1,
    max_mem_size=10
)

# 데이터 분리
X = X_ohe_encoded.drop(columns=['fog_train.vis1'])
y = X_ohe_encoded['fog_train.vis1']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 학습 및 테스트 데이터를 H2O 프레임으로 변환
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

# H2O 프레임으로 변환
train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

# 특성과 타겟 변수 설정
x = train_h2o.columns
y = 'fog_train.vis1'
x.remove(y)

# H2O AutoML 모델 학습
aml = H2OAutoML(max_runtime_secs=3600, seed=42, project_name='fog_train_regression')
aml.train(x=x, y=y, training_frame=train_h2o)

# 모델 리더보드 출력
lb = aml.leaderboard
print(lb)

# 베스트 모델
best_model = aml.leader
print(best_model)

# 테스트 데이터에 대한 예측
preds = best_model.predict(test_h2o)
print(preds)

# H2O 서버 종료
h2o.shutdown(prompt=False)

: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from imblearn.under_sampling import RandomUnderSampler

# 예측된 값으로부터 클래스 레이블 생성
y_class = (y_pred > 1000).astype(int)

# 데이터 분리
X_class = X_test
y_class = pd.Series(y_class, name='class')

# 클래스 불균형 처리
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_class, y_class)

# 데이터 분할
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# 모델 학습
classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_res, y_train_res)

# 예측
y_pred_res = classifier.predict(X_test_res)

# 모델 평가
accuracy = accuracy_score(y_test_res, y_pred_res)
f1 = f1_score(y_test_res, y_pred_res)

accuracy, f1


: 